In [1]:
# NLP Project
# Detecting Errors in Parsing of Speech Transcript
# Sandesh

In [2]:
import spacy
from spacy import displacy
import nltk
import numpy as np
import matplotlib.pyplot as plt
import warnings
import pandas as pd
from IPython.display import display
warnings.filterwarnings('ignore')

In [3]:
test_files = {}

for test_file in range(1, 5):
    file = open('test_' + str(test_file) + '.txt', 'r')
    file_string = file.read()
    tokenized_sentences = file_string.split('\n')
    test_files[test_file] = tokenized_sentences

In [4]:
sentence_example_A = test_files[1][1]
sentence_example_B = "I would like to , I would want to go to the zoo ."

In [5]:
# Initializing the processor
proc = spacy.load("en_core_web_sm")
embedder = spacy.load("en_core_web_lg")

# if errors: do this, this is what worked for me
# python -m spacy download en
# conda install -c conda-forge spacy
# python -m spacy download en_core_web_sm
# python -m spacy link en_core_web_sm en --force

In [6]:
extract = proc(sentence_example_B)
# uncomment the following line to see a version of the dependency graph
displacy.serve(extract, style="dep")


Using the 'dep' visualizer
Serving on http://0.0.0.0:5000 ...

Shutting down server on port 5000.


In [7]:
def find_root(extract):
    for token in extract:
        if token.head == token:
            return token
        # root has been found

In [8]:
def check_xcomp_violations(node):
    all_children = [child for child in node.children]
    
    if len(all_children) == 0:
        if node.dep_ == 'xcomp':
            return False
        else:
            return True
    
    else:
        subtree_result = True
        for child in node.children:
            subtree_result = subtree_result and check_xcomp_violations(child)
            
        return subtree_result

In [9]:
def check_xcomp_conditions(extract, suppress = 0):
    root_token = find_root(extract)
    is_valid = check_xcomp_violations(root_token)
    
    if is_valid == True:
        if suppress == 0:
            print("No violations in xcomp clause structures found")
        return 1
    else:
        if suppress == 0:
            print("xcomp clause structure violated.")
        return 0
        
    # returns code 1 for Green, 0 for Red

In [10]:
def get_size(node):
    all_children = [child for child in node.children]
    
    if len(all_children) == 0:
        # encountered a leaf node
        return 1
    else:
        subtree_size = 1  

        for child in node.children:
            subtree_size += get_size(child)
            
        return subtree_size

In [11]:
def check_connected(extract, suppress = 0):
    root_token = find_root(extract)
    tree_size = get_size(root_token)
    
    if suppress == 0:
        print("Number of Tokens: ", str(tree_size))
    if tree_size == len(extract):
        if suppress == 0:
            print("The Parse yields a tree.")
        return 1
    else:
        if suppress == 0:
            print("The parse does not yield a tree. Hence, this is an error.")
        return 0
        
    # returns code 1 for Green, 0 for Red

In [12]:
def display_features(extract):
    count = 0
    index_arr = []
    words = []
    labels = []
    parent_word = []
    pos = []
    count_children = []
    
    for token in extract:
        count += 1
        index_arr.append(str(count))
        words.append(token.text)
        labels.append(token.dep_)
        parent_word.append(token.head.text)
        pos.append(token.pos_.lower())
        count_children.append(len([child for child in token.children]))
        
    d = {'Token Word': words, 'Dependency Label': labels, 'Parent Word': parent_word,
         'POS': pos, 'Number of Children': count_children}
    df = pd.DataFrame(index = index_arr, data = d)
    display(df)

In [13]:
# Repetition Detection
# Based on word-embeddings
# Using embedder, loaded above
all_structs = []

def get_structs(node):
    all_children = [child for child in node.children]
    
    if len(all_children) == 0:
        # encountered a leaf node
        embeddings = embedder(node.text)
        create_struct = {'size': 1, 'hash': np.asarray(embeddings[0].vector), 'node': node} 
        all_structs.append(create_struct)
        return create_struct
    
    else:
        subtree_size = 1
        embeddings = embedder(node.text)
        subtree_hash = np.asarray(embeddings[0].vector)
        
        for child in node.children:
            temp_struct = get_structs(child)
            subtree_hash = subtree_hash + temp_struct['hash']
            subtree_size += temp_struct['size']
            
        create_struct = {'size': subtree_size, 'hash': subtree_hash, 'node': node}
        all_structs.append(create_struct)
        return create_struct
                
def make_all_structs(extract):
    global all_structs
    all_structs = []
    root = find_root(extract)
    # create a dictionary that returns hash_value and size of subtrees rooted at each node
    
    root_struct = get_structs(root)
    return root_struct

def check_if_subtree(struct_A, struct_B):
    if struct_A['size'] > struct_B['size']:
        temp_struct = struct_A
        struct_A = struct_B
        struct_B = temp_struct
        
    token_big = struct_B['node']
    token = struct_A['node']
    
    while token.head != token:
        if token.head == token_big:
            return True
        token = token.head
    
    return False

def get_lca(struct_A, struct_B):
    path_to_A = []
    path_to_B = []
    
    node_A = struct_A['node']
    node_B = struct_B['node']
    
    while(node_A.head != node_A):
        path_to_A.append(node_A)
        node_A = node_A.head
        
    while(node_B.head != node_B):
        path_to_B.append(node_B)
        node_B = node_B.head
        
    path_to_A.reverse()
    path_to_B.reverse()
    
    index = 0
    
    while True:
        if index >= len(path_to_A) or index >= len(path_to_B):
            break
        if path_to_A[index] != path_to_B[index]:
            break
        index += 1
        
    return index

def get_height(struct):
    node = struct['node']
    height = 0
    while(node.head != node):
        height += 1
        node = node.head
    
    return height

def get_distance(struct_A, struct_B):
    distance = get_height(struct_A) + get_height(struct_B) - 2 * get_lca(struct_A, struct_B)
    return distance

def similarity(vec_A, vec_B):
    score_cos = np.dot(vec_A, vec_B) / (np.linalg.norm(vec_A) * np.linalg.norm(vec_B))
    score_dist = np.linalg.norm(vec_A - vec_B)
    return {'cosine': score_cos, 'dist': score_dist}

def check_repetitions(suppress = 0):
    global all_structs
    count_A = 0
    for struct_A in all_structs:
        count_B = 0
        for struct_B in all_structs:
            if count_A == count_B:
                continue
            if (abs(count_A - count_B) <= 5) and get_distance(struct_A, struct_B) <= 7 and check_if_subtree(struct_A, struct_B) == False:
                results = similarity((struct_A['hash'] / struct_A['size']), (struct_B['hash'] / struct_B['size']))
                if results['cosine'] > 0.975 and results['dist'] < 1.15:
                    return 1
            count_B += 1
        count_A += 1
    return 0
    # if repetition was found, return a 1
    # else, return 0

In [14]:
# Repetition Detection
# Using N-sized window

window_size = 7

def check_repeat(extract, suppress = 0):
    memory = []
    repeats = 0
    
    for token in extract:
        embeddings = embedder(token.text)
        
        is_fine = 1
        for mem in memory:
            score = similarity(np.asarray(embedder(mem.text).vector), np.asarray(embeddings[0].vector))
            if score['cosine'] > 0.95 and (mem.pos_ == 'NOUN' or mem.pos_ == 'VERB' or token.pos_ == 'NOUN' or token.pos_ == 'VERB'):
                is_fine = 0
                break
        
        if is_fine == 0:
            repeats = 1
            break
        
        memory.append(token)
        
        while (len(memory) > window_size):
            rem = memory.pop()
    
    if repeats == 1:
        if suppress != 1:
            print("Lexical Repetition Encountered")
        return 1
    else:
        if suppress != 1:
            print("No Lexical Repetitons Encountered")
        return 0

In [15]:
# checks if a word-bigram has been uttered twice in atleast one window of 5 words
def check_bi_repeat(extract, suppress = 0):
    window = min(31, len(extract))
    memory = []
    for token in extract:
        memory.append(token.text)
        if(len(memory) < window):
            continue
        
        while len(memory) > window:
            rem = memory.pop()
            
        for init in range(2, window - 1):
            if memory[0] == memory[init] and memory[1] == memory[init + 1]:
                if suppress != 1:
                    print("Bigram repeated")
                return 1
    
    if suppress != 1:
        print("No bigram repetitions found")
    return 0

In [16]:
# USAGE:
print("Number of Tokens: ", len(extract))

# Visualize the set of the extracted features
display_features(extract)

# if the resultant parse tree is connected or not
temp = check_connected(extract)

# if the resultant parse tree has any violations in xcomp structure
temp = check_xcomp_conditions(extract)

# get the tree-hash of the dependency tree
temp = make_all_structs(extract)

# check for lexical repetitions in 5 word windows
result = check_repeat(extract)

# check for repetitions of "I" in 5 word windows
check = check_bi_repeat(extract)

Number of Tokens:  14


,Token Word,Dependency Label,Parent Word,POS,Number of Children
1,I,nsubj,like,pron,0
2,would,aux,like,aux,0
3,like,ccomp,want,verb,3
4,to,xcomp,like,part,0
5,",",punct,want,punct,0
6,I,nsubj,want,pron,0
7,would,aux,want,aux,0
8,want,ROOT,want,verb,6
9,to,aux,go,part,0
10,go,xcomp,want,verb,2


Number of Tokens:  14
The Parse yields a tree.
xcomp clause structure violated.
No Lexical Repetitons Encountered
Bigram repeated


In [18]:
# Analysis:
test_file_indices = [n for n in range(1, 5)]

sentence_count = 0
not_a_tree = 0
xcomp_violated = 0
repeated_count = 0
repeated_count_parse = 0
bi_count = 0
invalid_count = 0

invalid_repeated = 0
invalid_repeated_parse = 0

invalid_repeated_sentences = []
invalid_repeated_parse_sentences = []

for idx in test_file_indices:
    sentences = test_files[idx]

    for sentence in sentences:
        sentence_count += 1
        
        if(sentence_count % 43 == 0):
            print(str(round(sentence_count / 865 * 100)) + "% Completed")

        extract_info = proc(sentence)

        temp = make_all_structs(extract_info)
        is_repetition_parse = check_repetitions(1)
        
        is_repetition_window = check_repeat(extract_info, 1)

        is_bi_repeat = check_bi_repeat(extract_info, 1)
        
        flag_valid = 1
        
        if check_connected(extract_info, 1) == 0:
            not_a_tree += 1
            flag_valid = 0
            
        if check_xcomp_conditions(extract_info, 1) == 0:
            xcomp_violated += 1
            flag_valid = 0
            
        if flag_valid != 1:
            invalid_count += 1
        
        repeated_count += is_repetition_window
        repeated_count_parse += is_repetition_parse

        bi_count += is_bi_repeat
        
        if flag_valid == 0 and is_repetition_window == 1:
            invalid_repeated += 1
            invalid_repeated_sentences.append(sentence)
            
        if flag_valid == 0 and is_repetition_parse == 1:
            invalid_repeated_parse += 1
            invalid_repeated_parse_sentences.append(sentence)
        
print("Done")

5% Completed
10% Completed
15% Completed
20% Completed
25% Completed
30% Completed
35% Completed
40% Completed
45% Completed
50% Completed
55% Completed
60% Completed
65% Completed
70% Completed
75% Completed
80% Completed
85% Completed
89% Completed
94% Completed
99% Completed
Done


In [19]:
# Displaying the Results
index_arr = ['Sentence Count', 'Not a Tree', '\'xcomp\' Condition Violated', 'Possible Repetitions using Window', 'Possible Repetitions using Parse',
             'Bigrams repeated', 'Parsing Constraint violated', 'Violation + Repetition with Window', 'Violation + Repetition with Parse']
results_arr = [sentence_count, not_a_tree, xcomp_violated, repeated_count, repeated_count_parse, bi_count,
               invalid_count, invalid_repeated, invalid_repeated_parse]
d_results = {"Results": results_arr}
df_results = pd.DataFrame(index = index_arr, data = d_results)
display(df_results)

,Results
Sentence Count,865
Not a Tree,40
'xcomp' Condition Violated,2
Possible Repetitions using Window,253
Possible Repetitions using Parse,627
Bigrams repeated,2
Parsing Constraint violated,42
Violation + Repetition with Window,18
Violation + Repetition with Parse,30


In [20]:
print("Percentage of Sentences violating parsing constraints: "
      + str(round(invalid_count / sentence_count * 100, 2)) + '%')
print("Percentage of wrongly parsed Sentences showing Repetitions based on Parse: "
      + str(round(invalid_repeated_parse / invalid_count * 100, 2)) + '%')
print("Percentage of wrongly parsed Sentences showing Repetitions based on Window: "
      + str(round(invalid_repeated / invalid_count * 100, 2)) + '%')

Percentage of Sentences violating parsing constraints: 4.86%
Percentage of wrongly parsed Sentences showing Repetitions based on Parse: 71.43%
Percentage of wrongly parsed Sentences showing Repetitions based on Window: 42.86%


In [21]:
# Visualizing results qualitatively

print("Invalid Sentences with Possibility of repetition using window:")
print(invalid_repeated_sentences)

print("Invalid Sentences with Possibility of repetition using parse:")
print(invalid_repeated_parse_sentences)

Invalid Sentences with Possibility of repetition using window:
['Now , you can invoke free vortex law , either at the mean of the rotor blade or you may like to invoke it at the exit of the rotor blade , you can theoretically invoke it at the entry to the rotor blade , but normally it is not done , because the free vortex characteristic is acquired by the flow only when the flow goes to the rotor .', 'Now , this is of course , of value U , which you get from solid body relationship and that is the blade speed at that particular section U 1 r . Correspondingly , now you can find out the relative flow angle going into the blade and this of course , as we know , is found from the velocity triangle .', 'Now , the exit angle , flow angles can be also found using the whirl component and the axial components and the relative flow angle can also be found by using the blade velocity , the whirl component and the axial component by using the simple trigonometric relationship and if you do that ,

In [22]:
# That's All Folks